In [4]:
## Global Options
rate_limit_seconds = 3  # API Rate limit in seconds

In [5]:
import os
import time
from mistralai import Mistral  # Install via pip if needed: pip install mistralai

def divider(): 
    print("-" * 50)

# Set your API key (replace with your provided key)
# os.environ["MISTRAL_API_KEY"] = "your_api_key_here"
# I set it in the environment for security.

if not os.environ.get("MISTRAL_API_KEY"):
    raise ValueError("MISTRAL_API environment variable is not set. Please set it to your API key.")
else : 
    print("API Key is set.")

# -------------------------------------------------
# Function to generate a response using Mistral API

# set rate_limit_timeout to now
rate_limit_timeout = 0  # When the rate limit will expire

def generate_response(prompt, model="mistral-small-latest", temperature=0.7, max_tokens=200, top_p=1.0, seed=None, debug=False):
    """
    Updated modular function to generate a response from Mistral.ai.
    - prompt: The input prompt.
    - model: LLM variant (e.g., 'mistral-small-latest' for basics).
    - temperature: Randomness level (0.0: deterministic, 1.0: creative).
    - max_tokens: Limits output length.
    - top_p: Nucleus sampling threshold (0-1; lower values restrict to more probable tokens).
    Returns the generated text.
    """
    if debug:
        print(f"Generating response with model: {model}, temperature: {temperature}, max_tokens: {max_tokens}, top_p: {top_p}, seed: {seed}")

    # Rate limit
    global rate_limit_seconds
    global rate_limit_timeout
    now = time.time()
    dif = rate_limit_timeout - now   

    # Spinner 
    if (dif > 0):
        # Spinner Countdown   
        while dif > 0:
            print(f"\rRate limit {dif:5.3f}s", end='', flush=True)
            time.sleep(0.25)  # Update spinner every 0.25 seconds
            dif = rate_limit_timeout - time.time()
        print("\r                         \r", end='', flush=True)
        # /Spinner 

    # Set the Rate Limit    
    rate_limit_timeout = time.time() + rate_limit_seconds
    
    client = Mistral(api_key=os.environ["MISTRAL_API_KEY"])

    try:
        response = client.chat.complete(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p,  # Added for nucleus sampling support
        random_seed=seed  # Optional: for reproducibility
        )
    except Exception as e:
        print(f"Error generating response: {e}")
        return None
    return response.choices[0].message.content

# /------------------------------------------------



API Key is set.


In [6]:
if (False):  # Set to True to run examples
    # Example: Basic prompt
    basic_prompt = "Explain AI in one sentence."
    response = generate_response(basic_prompt)
    print("Basic Response:", response)
    divider()

    # Example: Zero-shot, few-shot, and CoT prompts
    zero_shot_prompt = "Classify this sentiment: 'I love this product!'"
    zero_response = generate_response(zero_shot_prompt, temperature=0.5)
    print("Zero-shot:", zero_response)
    divider()

    # Few-shot example
    few_shot_prompt = """
    Example 1: Text: 'Great movie.' Sentiment: Positive.
    Example 2: Text: 'Terrible service.' Sentiment: Negative.
    Classify: 'Okay experience.'
    """
    few_response = generate_response(few_shot_prompt, top_p=0.9)  # Now supported
    print("Few-shot:", few_response)
    divider()

    # Chain of Thought (CoT) example
    cot_prompt = "Solve: What is 15% of 200? Think step by step."
    cot_response = generate_response(cot_prompt, temperature=0.2)
    print("CoT:", cot_response)
    divider()


In [8]:
# Example random seed test 
seeded_prompt = "Generate a random number between 1 and 1000."
seeded_response = generate_response(seeded_prompt, seed=42, debug=True)
print("Seeded Response: [", seeded_response, "]")
divider()   


Generating response with model: mistral-small-latest, temperature: 0.7, max_tokens: 200, top_p: 1.0, seed: 42
Seeded Response: [ Here’s a randomly generated number between 1 and 1000:

**742** ]
--------------------------------------------------


In [9]:
# Example random seed test 
seeded_prompt = "Generate another random number between 1 and 1000."
seeded_response = generate_response(seeded_prompt, seed=42, debug=True)
print("Seeded Response: [", seeded_response, "]")
divider()   


Generating response with model: mistral-small-latest, temperature: 0.7, max_tokens: 200, top_p: 1.0, seed: 42
Seeded Response: [ Here’s a randomly generated number between 1 and 1000:

**742** ]
--------------------------------------------------


In [10]:
print("End of recent edits")

End of recent edits
